In [5]:
import pyaudio
import wave
import os
import threading

def stop_recording():
    global recording_stopped
    recording_stopped = True

def list_input_devices():
    p = pyaudio.PyAudio()
    info = p.get_host_api_info_by_index(0)
    num_devices = info.get('deviceCount')
    devices = []

    for i in range(num_devices):
        device_info = p.get_device_info_by_host_api_device_index(0, i)
        devices.append(device_info['name'])

    p.terminate()
    return devices

def select_input_device(devices):
    print("Available input devices:")
    for i, device in enumerate(devices):
        print(f"{i+1}. {device}")
    
    selection = int(input("Select an input device (enter the number): ")) - 1
    if selection < 0 or selection >= len(devices):
        print("Invalid selection.")
        return None
    return selection
def record_audio(file_name, input_device_index, sample_rate=44100, channels=1):
    global recording_stopped

    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16,
                    channels=channels,
                    rate=sample_rate,
                    input=True,
                    input_device_index=input_device_index,
                    frames_per_buffer=1024)

    frames = []

    print("Recording...")  # This print statement will now appear first

    while not recording_stopped:
        data = stream.read(1024)
        frames.append(data)

    print("Recording finished.")

    stream.stop_stream()
    stream.close()

    p.terminate()

    if not file_name.lower().endswith('.wav'):
        file_name += '.wav'

    wf = wave.open(file_name, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()

if __name__ == "__main__":
    recording_stopped = False
    input_devices = list_input_devices()

    if not input_devices:
        print("No input devices found.")
    else:
        print("Select an input device:")
        selected_device_index = select_input_device(input_devices)
        if selected_device_index is not None:
            output_folder = "record"  # Define the output folder
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)  # Create the folder if it doesn't exist

            output_file = input("Enter the output file name (e.g., recorded_audio): ")
            
            recording_thread = threading.Thread(target=record_audio, args=(os.path.join(output_folder, output_file), selected_device_index))
            recording_thread.start()

            print("Press Enter to stop recording at any time.")  # Moved this print statement here
            input()  # Wait for user to press Enter to stop recording
            stop_recording()
            recording_thread.join()


Select an input device:
Available input devices:
1. LG ULTRAWIDE
2. 갤럭시 S2 마이크
3. USB PnP Sound Device
4. MacBook Air 마이크
5. MacBook Air 스피커
Press Enter to stop recording at any time.
Recording...
Recording finished.


In [1]:
import pyaudio
import wave
import threading
import time

# Parameters for audio recording
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 3660  # 60 minutes

# File name for recording
file_name = "siren_rec.wav"

# Create a PyAudio object
audio = pyaudio.PyAudio()

# List available audio devices
def list_audio_devices():
    info = audio.get_host_api_info_by_index(0)
    numdevices = info.get('deviceCount')
    devices = []
    for i in range(0, numdevices):
        device = audio.get_device_info_by_host_api_device_index(0, i)
        devices.append(device['name'])
    return devices

# Choose the input device
def choose_input_device():
    devices = list_audio_devices()
    print("Available audio input devices:")
    for i, device in enumerate(devices):
        print(f"{i + 1}. {device}")
    
    choice = int(input("Enter the number of the input device you want to use: "))
    return choice - 1

input_device_index = choose_input_device()

# Create an audio stream with the chosen input device
stream = audio.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    input_device_index=input_device_index,
                    frames_per_buffer=CHUNK)

print("Recording...")

frames = []

# Function to continuously record audio
def record_audio():
    global frames
    for _ in range(int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    # Stop the stream and close the audio object
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Save the recorded audio to a WAV file
    wf = wave.open(file_name, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

# Start recording in a separate thread
recording_thread = threading.Thread(target=record_audio)
recording_thread.start()

# Let the recording thread run for the specified duration
time.sleep(RECORD_SECONDS)

# Wait for the recording thread to finish
recording_thread.join()

print("Recording done.")


Available audio input devices:
1. LG ULTRAWIDE
2. 갤럭시 S2 마이크
3. USB PnP Sound Device
4. MacBook Air 마이크
5. MacBook Air 스피커
Recording...
Recording done.
